In [7]:
import pandas as pd
import numpy as np
from supabase import create_client
import json

# ... (credentials remain the same)

def clean_chunk(df):
    # Convert the entire dataframe to object type to prevent pandas from 
    # trying to be "smart" with float optimization during the conversion
    df = df.astype(object)
    
    # Explicitly replace all non-finite values (NaN, Inf, -Inf) with None
    # This works column by column
    for col in df.columns:
        # Check if the column is numeric-like
        if df[col].dtype == object:
            # Replace common problematic values
            df.loc[~df[col].apply(lambda x: isinstance(x, (str, int, float)) or x is None), col] = None
            # If it is a float, check if it's finite
            df.loc[df[col].apply(lambda x: isinstance(x, float) and not np.isfinite(x)), col] = None
            
    # Final pass to ensure all NaNs are None
    return df.where(pd.notnull(df), None)

def upload_data(file_name):
    chunk_size = 2000 # Smaller chunks are safer for debugging 1M rows
    print(f"Reading {file_name}...")
    
    # Open file with low_memory=False to ensure types are consistent
    for i, chunk in enumerate(pd.read_csv(file_name, chunksize=chunk_size, low_memory=False)):
        cleaned_data = clean_chunk(chunk).to_dict(orient='records')
        
        try:
            # We use json.dumps to "test" the chunk before sending it
            # This identifies EXACTLY if the error is local
            json_test = json.dumps(cleaned_data)
            
            supabase.table("trips").insert(cleaned_data).execute()
            print(f"✅ Chunk {i+1} uploaded (Total: {(i+1) * chunk_size})")
        except (ValueError, Exception) as e:
            print(f"❌ Failed at chunk {i+1}: {e}")
            break

if __name__ == "__main__":
    upload_data('citibike_2025_combined.csv')

Reading citibike_2025_combined.csv...
✅ Chunk 1 uploaded (Total: 2000)
✅ Chunk 2 uploaded (Total: 4000)
✅ Chunk 3 uploaded (Total: 6000)
✅ Chunk 4 uploaded (Total: 8000)
✅ Chunk 5 uploaded (Total: 10000)
✅ Chunk 6 uploaded (Total: 12000)
✅ Chunk 7 uploaded (Total: 14000)
✅ Chunk 8 uploaded (Total: 16000)
✅ Chunk 9 uploaded (Total: 18000)
✅ Chunk 10 uploaded (Total: 20000)
✅ Chunk 11 uploaded (Total: 22000)
✅ Chunk 12 uploaded (Total: 24000)
✅ Chunk 13 uploaded (Total: 26000)
✅ Chunk 14 uploaded (Total: 28000)
✅ Chunk 15 uploaded (Total: 30000)
✅ Chunk 16 uploaded (Total: 32000)
✅ Chunk 17 uploaded (Total: 34000)
✅ Chunk 18 uploaded (Total: 36000)
✅ Chunk 19 uploaded (Total: 38000)
✅ Chunk 20 uploaded (Total: 40000)
✅ Chunk 21 uploaded (Total: 42000)
✅ Chunk 22 uploaded (Total: 44000)
✅ Chunk 23 uploaded (Total: 46000)
✅ Chunk 24 uploaded (Total: 48000)
✅ Chunk 25 uploaded (Total: 50000)
✅ Chunk 26 uploaded (Total: 52000)
✅ Chunk 27 uploaded (Total: 54000)
✅ Chunk 28 uploaded (Total: 56

KeyboardInterrupt: 